In [ ]:
https://zhuanlan.zhihu.com/p/255135694

In [9]:
import numpy as np
#二叉树模型对欧式看涨期权定价
def binarytree_europcall(S,K,r,q,sigma,t,steps):
    '''S:标的资产初始价格；
    K:期权的执行价格;
    r:年化无风险利率；
    q:连续分红的红利率；
    sigma:标的资产连续复利收益率的标准差；
    t:以年表示的时间长度；
    steps:二叉树的步长。'''
    u=np.exp(sigma*np.sqrt(t/steps))   #时间间隔为△t=t/steps
    d=1/u     
    P=(np.exp((r-q)*t/steps)-d)/(u-d)
    #基础公式
    
    prices=np.zeros(steps+1)       #生成最后一列的股票价格空数组
    c_values=np.zeros(steps+1)     #生成最后一列的期权价值空数组
    #0数组：steps=2时，是一个三个元素都是0的 array
    
    prices[0]=S*d**steps           #最后一行最后一列的股票价格
    c_values[0]=np.maximum(prices[0]-K,0)#最后一行最后一列的期权价值
    #prices[0]=663.17
    #c_value[0]=0
    
    for i in range(1,steps+1):
        prices[i]=prices[i-1]*(u**2)   #计算最后一列的股票价格
        c_values[i]=np.maximum(prices[i]-K,0)  #计算最后一列的期权价值
        
    #prices[1]=prices[0]*(u**2)=sdd*uu = sdd*(1/d)*(1/d)=s=810
    #prices[2]=prices[1]*(u**2)=suu =989.34    
        
    for j in range(steps,0,-1):       
    #逐个节点往前计算
#range(2,0,-1) 即(2,1,0) range的第三个参数是间隔切不包括第三个数 即仅有2，1
        for i in range(0,j):
#当j=2时，range(0,j)=range(0,2):计算期权价格的第二列
#c_values[0]=5.06 / c_value[1]=100.66
#当j=1时，range(0,j)=range(0,1）:计算期权价格的第一列
#c_values[0]=53.4

#期权从后往前推：P*c_values[i+1]+(1-P)*c_values[i])*np.exp(-r*t/steps
#     （股票涨概率 * 涨了之后的期权价格 +股票跌概率 * 跌了之后的期权价格）折现

            c_values[i]=(P*c_values[i+1]+(1-P)*c_values[i])*np.exp(-r*t/steps)
    
    return c_values[0]

A=binarytree_europcall(S=810,K=800,r=0.05,q=0.02,sigma=0.2,t=0.5,steps=2)
print('运用二叉树模型计算欧式看涨期权的价格为：',round(A,1))

运用二叉树模型计算欧式看涨期权的价格为： 53.4


In [5]:
#二叉树模型对美式看跌期权定价
def binarytree_americaput(S,K,r,q,sigma,t,steps):
    '''S:标的资产初始价格；
    K:期权的执行价格;
    r:年化无风险利率；
    q:连续分红的红利率；
    sigma:标的资产连续复利收益率的标准差；
    t:以年表示的时间长度；
    steps:二叉树的步长。'''
    u=np.exp(sigma*np.sqrt(t/steps))
    d=1/u
    P=(np.exp((r-q)*t/steps)-d)/(u-d)
    prices=np.zeros(steps+1)
    c_values=np.zeros(steps+1)
    prices[0]=S*d**steps
    c_values[0]=np.maximum(K-prices[0],0)
    for i in range(1,steps+1):
        prices[i]=prices[i-1]*(u**2)
        c_values[i]=np.maximum(K-prices[i],0)
        
    for j in range(steps,0,-1):
        for i in range(0,j):
            prices[i]=prices[i+1]*d #就这里不一样，要算出执行价差于欧式期权的现值对比，如果超过欧式期权折现值就要提前执行
            c_values[i]=np.maximum((P*c_values[i+1]+(1-P)*c_values[i])*np.exp(-r*t/steps),K-prices[i])
    return c_values[0]

C=binarytree_americaput(S=29,K=30,r=0.03,q=0,sigma=0.25,t=1,steps=2)
print('运用二叉树模型计算美式看跌期权的价格为：',round(C,3))

运用二叉树模型计算美式看跌期权的价格为： 3.058


In [17]:
#欧式看涨自己打的练习

import numpy as np
def binarytree_europcall(S,K,r,q,sigma,t,steps):
    u = np.exp(sigma*(t/steps)**(1/2))
    d = 1/u
    P = (np.exp((r-q)*(t/steps))-d)/(u-d)
     
    prices=np.zeros(steps+1)
    c_values=np.zeros(steps+1)
    
    prices[0]=S*d**steps
    c_values[0]=np.maximum(prices[0]-K,0)
    #可以先不算期权最后一行最后一列的价格
    
    for i in range(1,steps+1):
        prices[i]=prices[i-1]*(u**2)
        #c_values[i]=np.maximum(prices[i]-K,0)
        #此处仅构建股票最后一列前两行的价格
        
#如果不要先算期权的最后一行的最后一列，可以额外构建一个for循环
#此处先算是因为最后一列的前两行，即c_values[1,2]的构建方法和股票价格相同

    for i in range(0,steps+1):
       c_values[i]=np.maximum(prices[i]-K,0)
        #期权最后一列的价格
        
    for j in range(steps,0,-1):
        for i in range(0,j):
            prices[i]=prices[i]*u
            c_values[i]=((1-P)*c_values[i]+P*c_values[i+1])*(np.exp(-r*t/steps))
            
    return c_values[0]


            
A=binarytree_europcall(S=810,K=800,r=0.05,q=0.02,sigma=0.2,t=0.5,steps=2)
print('运用二叉树模型计算欧式看涨期权的价格为：',round(A,1))
            
    
    
    
    

运用二叉树模型计算欧式看涨期权的价格为： 53.4
